In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import pretty_midi


In [12]:
import shutil
def save_ckp(state, is_best, checkpoint_dir, best_model_dir):
    f_path = checkpoint_dir / 'checkpoint.pt'
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir / 'best_model.pt'
        shutil.copyfile(f_path, best_fpath)
        
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    #optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']

In [3]:
#saving array to midi and stuff
def set_piano_roll_to_instrument(piano_roll, instrument, velocity=100, tempo=120.0, beat_resolution=16):
    # Calculate time per pixel
    tpp = 60.0 / tempo / float(beat_resolution)
    threshold = 60.0 / tempo / 4
    phrase_end_time = 60.0 / tempo * 4 * piano_roll.shape[0]
    # Create piano_roll_search that captures note onsets and offsets
    piano_roll = piano_roll.reshape((piano_roll.shape[0] * piano_roll.shape[1], piano_roll.shape[2]))
    piano_roll_diff = np.concatenate((np.zeros((1, 128), dtype=int), piano_roll, np.zeros((1, 128), dtype=int)))
    piano_roll_search = np.diff(piano_roll_diff.astype(int), axis=0)
    # Iterate through all possible(128) pitches

    for note_num in range(128):
        # Search for notes
        start_idx = (piano_roll_search[:, note_num] > 0).nonzero()
        start_time = list(tpp * (start_idx[0].astype(float)))
        # print('start_time:', start_time)
        # print(len(start_time))
        end_idx = (piano_roll_search[:, note_num] < 0).nonzero()
        end_time = list(tpp * (end_idx[0].astype(float)))
        # print('end_time:', end_time)
        # print(len(end_time))
        duration = [pair[1] - pair[0] for pair in zip(start_time, end_time)]
        # print('duration each note:', duration)
        # print(len(duration))

        temp_start_time = [i for i in start_time]
        temp_end_time = [i for i in end_time]

        for i in range(len(start_time)):
            # print(start_time)
            if start_time[i] in temp_start_time and i != len(start_time) - 1:
                # print('i and start_time:', i, start_time[i])
                t = []
                current_idx = temp_start_time.index(start_time[i])
                for j in range(current_idx + 1, len(temp_start_time)):
                    # print(j, temp_start_time[j])
                    if temp_start_time[j] < start_time[i] + threshold and temp_end_time[j] <= start_time[i] + threshold:
                        # print('popped start time:', temp_start_time[j])
                        t.append(j)
                        # print('popped temp_start_time:', t)
                for _ in t:
                    temp_start_time.pop(t[0])
                    temp_end_time.pop(t[0])
                # print('popped temp_start_time:', temp_start_time)

        start_time = temp_start_time
        # print('After checking, start_time:', start_time)
        # print(len(start_time))
        end_time = temp_end_time
        # print('After checking, end_time:', end_time)
        # print(len(end_time))
        duration = [pair[1] - pair[0] for pair in zip(start_time, end_time)]
        # print('After checking, duration each note:', duration)
        # print(len(duration))

        if len(end_time) < len(start_time):
            d = len(start_time) - len(end_time)
            start_time = start_time[:-d]
        # Iterate through all the searched notes
        for idx in range(len(start_time)):
            if duration[idx] >= threshold:
                # Create an Note object with corresponding note number, start time and end time
                note = pretty_midi.Note(velocity=velocity, pitch=note_num, start=start_time[idx], end=end_time[idx])
                # Add the note to the Instrument object
                instrument.notes.append(note)
            else:
                if start_time[idx] + threshold <= phrase_end_time:
                    # Create an Note object with corresponding note number, start time and end time
                    note = pretty_midi.Note(velocity=velocity, pitch=note_num, start=start_time[idx],
                                            end=start_time[idx] + threshold)
                else:
                    # Create an Note object with corresponding note number, start time and end time
                    note = pretty_midi.Note(velocity=velocity, pitch=note_num, start=start_time[idx],
                                            end=phrase_end_time)
                # Add the note to the Instrument object
                instrument.notes.append(note)
    # Sort the notes by their start time
    instrument.notes.sort(key=lambda note: note.start)
    # print(max([i.end for i in instrument.notes]))
    # print('tpp, threshold, phrases_end_time:', tpp, threshold, phrase_end_time)

def write_piano_rolls_to_midi(piano_rolls, program_nums=None, is_drum=None, filename='test.mid', velocity=100,
                              tempo=120.0, beat_resolution=24):
    if len(piano_rolls) != len(program_nums) or len(piano_rolls) != len(is_drum):
        print("Error: piano_rolls and program_nums have different sizes...")
        return False
    if not program_nums:
        program_nums = [0, 0, 0]
    if not is_drum:
        is_drum = [False, False, False]
    # Create a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI(initial_tempo=tempo)
    # Iterate through all the input instruments
    for idx in range(len(piano_rolls)):
        # Create an Instrument object
        instrument = pretty_midi.Instrument(program=program_nums[idx], is_drum=is_drum[idx])
        # Set the piano roll to the Instrument object
        set_piano_roll_to_instrument(piano_rolls[idx], instrument, velocity, tempo, beat_resolution)
        # Add the instrument to the PrettyMIDI object
        midi.instruments.append(instrument)
    # Write out the MIDI data
    midi.write(filename)

def save_midis(bars, file_path, tempo=80.0):
    padded_bars = np.concatenate((np.zeros((bars.shape[0], bars.shape[1], 24, bars.shape[3])), bars,
                                  np.zeros((bars.shape[0], bars.shape[1], 20, bars.shape[3]))), axis=2)
    pause = np.zeros((bars.shape[0], 64, 128, bars.shape[3]))
    images_with_pause = padded_bars
    images_with_pause = images_with_pause.reshape(-1, 64, padded_bars.shape[2], padded_bars.shape[3])
    images_with_pause_list = []
    for ch_idx in range(padded_bars.shape[3]):
        images_with_pause_list.append(images_with_pause[:, :, :, ch_idx].reshape(images_with_pause.shape[0],
                                                                                 images_with_pause.shape[1],
                                                                                 images_with_pause.shape[2]))
    # write_midi.write_piano_rolls_to_midi(images_with_pause_list, program_nums=[33, 0, 25, 49, 0],
    #                                      is_drum=[False, True, False, False, False], filename=file_path, tempo=80.0)
    write_piano_rolls_to_midi(images_with_pause_list, program_nums=[0], is_drum=[False], filename=file_path,
                                         tempo=tempo, beat_resolution=4)

In [4]:
#loading data and stuff
def npy_loader(path):
    sample = torch.from_numpy(np.load(path))
    return sample

#X_train DATASET, X_TEST DATASET
X_TRAIN = datasets.DatasetFolder(
    root='classic_piano_x_train',
    loader=npy_loader,
    extensions=(".npy")
)

X_TEST = datasets.DatasetFolder(
    root='classic_piano_x_test',
    loader=npy_loader,
    extensions=(".npy")
)

Y_TRAIN = datasets.DatasetFolder(
    root='jazz_piano_y_train',
    loader=npy_loader,
    extensions=(".npy")
)

Y_TEST = datasets.DatasetFolder(
    root='jazz_piano_y_test',
    loader=npy_loader,
    extensions=(".npy")
)

dataloader_X= DataLoader(dataset=X_TRAIN, batch_size=16, shuffle=True, num_workers=0)
test_dataloader_X = DataLoader(dataset=X_TEST, batch_size=16, shuffle=True, num_workers=0)

dataloader_Y = DataLoader(dataset=Y_TRAIN, batch_size=16, shuffle=True, num_workers=0)
test_dataloader_Y = DataLoader(dataset=Y_TEST, batch_size=16, shuffle=True, num_workers=0)

In [5]:
#CONV helper function
def conv(in_channels, out_channels, kernel_size, stride, padding=1, instance_norm=False):
    """Creates a convolutional layer, with optional Instance normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels, out_channels, 
                           kernel_size, stride, padding, bias=False)
    
    # append conv layer
    layers.append(conv_layer)

    if instance_norm:
        # append batchnorm layer
        layers.append(nn.InstanceNorm2d(out_channels))
     
    # using Sequential container
    return nn.Sequential(*layers)

In [6]:
#DECONV helper function
def deconv(in_channels, out_channels, kernel_size, stride=2, padding=1,output_padding=1, instance_norm=False):
    """Creates a transposed-convolutional layer, with optional instance normalization.
    """
    # create a sequence of transpose + optional batch norm layers
    layers = []
    transpose_conv_layer = nn.ConvTranspose2d(in_channels, out_channels, 
                                              kernel_size, stride, padding,output_padding, bias=False)
    # append transpose convolutional layer
    layers.append(transpose_conv_layer)
    
    if instance_norm:
        # append batchnorm layer
        layers.append(nn.InstanceNorm2d(out_channels))
        
    return nn.Sequential(*layers)

In [11]:
#2 Generators 
#A-B generator and B-A generator
#1 discriminator associated with each generator
#A-B Discriminator and B-A Discriminator

class Discriminator(nn.Module):
    def __init__(self,conv_dim=64):
        
        super(Discriminator,self).__init__()
        
        self.conv_dim = conv_dim
        
        self.conv_layer_1 = conv(1,out_channels=conv_dim,kernel_size=4,stride=2,padding=1, instance_norm=False)
        
        self.conv_layer_2 = conv(conv_dim,out_channels=conv_dim*4,kernel_size=4,stride=2,padding=1, instance_norm=True)
        
        self.conv_layer_3 = conv(conv_dim*4,out_channels=1,kernel_size=1,stride=1,padding=0, instance_norm=False)
    
    def forward(self,x):
        
        output = F.leaky_relu(self.conv_layer_1(x),0.2)
        
        output = F.leaky_relu(self.conv_layer_2(output),0.2)
        
        output = self.conv_layer_3(output)
       
        return output
        

# discrim = Discriminator(64)
# print(discrim)
       

# data = torch.randn(1,1,64,84)
# print(discrim(data))

In [7]:
# residual block class
class ResidualBlock(nn.Module):
    """Defines a residual block.
       This adds an input x to a convolutional layer (applied to x) with the same size input and output.
       These blocks allow a model to learn an effective transformation from one domain to another.
    """
    def __init__(self, conv_dim=256):
        super(ResidualBlock, self).__init__()
        # conv_dim = number of inputs
        
        # define two convolutional layers + batch normalization that will act as our residual function, F(x)
        # layers should have the same shape input as output; I suggest a kernel_size of 3
        
        self.conv_layer1 = conv(in_channels=conv_dim, out_channels=conv_dim, 
                                kernel_size=3, stride=1, padding=1, instance_norm=True)
        
        self.conv_layer2 = conv(in_channels=conv_dim, out_channels=conv_dim, 
                               kernel_size=3, stride=1, padding=1, instance_norm=True)
        
    def forward(self, x):
       
        out_1 = F.relu(self.conv_layer1(x))
       
        out_2 = F.relu(x + self.conv_layer2(out_1))
        
        out_3 = x+out_2
        return out_3

In [8]:
def Res_layers(number,conv_dim=256):
    layers = []
    for i in range(number):
        layers.append(ResidualBlock(conv_dim))
        
    return nn.Sequential(*layers)
        

In [9]:
class Generator(nn.Module):
    def __init__(self,res_blocks,conv_dim=64):
        
        super(Generator,self).__init__()
        
        self.conv_dim = conv_dim
        
        self.conv_layer_1 = conv(1,self.conv_dim,kernel_size=7,stride=1,padding=3,instance_norm=True)
      
        
        self.conv_layer_2 = conv(conv_dim,self.conv_dim*2,kernel_size=3,stride=2,padding=1,instance_norm=True)
        
        
        self.conv_layer_3 = conv(self.conv_dim*2,self.conv_dim*4,kernel_size=3,stride=2,padding=1,instance_norm=True)
        
        
        self.res_layers = Res_layers(res_blocks,256)
        
        self.deconv_layer_1 = deconv(self.conv_dim*4,self.conv_dim*2,kernel_size=3,stride=2,padding=1,instance_norm=True)
        
        self.deconv_layer_2 = deconv(self.conv_dim*2,self.conv_dim,kernel_size=3,stride=2,padding=1,instance_norm=True)
        
        self.deconv_layer_3= deconv(self.conv_dim,1,kernel_size=7,stride=1,padding=3,output_padding=0,instance_norm=False)
        
    
    def forward(self,x):
        
        x = F.relu(self.conv_layer_1(x))
       
        x = F.relu(self.conv_layer_2(x))
        
        x = F.relu(self.conv_layer_3(x))
        
        x = self.res_layers(x)
       
        x = F.relu(self.deconv_layer_1(x))
        
        x = F.relu(self.deconv_layer_2(x))
        
        x = F.sigmoid(self.deconv_layer_3(x))
        
        
        return x
            
            
        
        
        
        

In [15]:
def create_model(g_conv_dim=64, d_conv_dim=64, n_res_blocks=10):
    """Builds the generators and discriminators."""
    
    # Instantiate generators
    G_XtoY = Generator(res_blocks=n_res_blocks,conv_dim=g_conv_dim)
    G_YtoX = Generator(res_blocks=n_res_blocks,conv_dim=g_conv_dim)
    # Instantiate discriminators
    D_X = Discriminator(conv_dim=d_conv_dim)
    D_Y = Discriminator(conv_dim=d_conv_dim)

    # move models to GPU, if available
    if torch.cuda.is_available():
        device = torch.device("cpu")
        G_XtoY.to(device)
        G_YtoX.to(device)
        D_X.to(device)
        D_Y.to(device)
        print('Models moved to CPU.')
    else:
        print('Only CPU available.')

    return G_XtoY, G_YtoX, D_X, D_Y

In [16]:
G_XtoY, G_YtoX, D_X, D_Y = create_model()

Models moved to CPU.


In [1]:
# from pytorch_modelsize import SizeEstimator

# se = SizeEstimator(D_X.to("cpu"), input_size=(16,1,64,84))
# print(se.estimate_size())



In [72]:
print(D_X)

Discriminator(
  (conv_layer_1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  )
  (conv_layer_2): Sequential(
    (0): Conv2d(64, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  )
  (conv_layer_3): Sequential(
    (0): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
)


In [17]:
#losses
def real_mse_loss(D_out):
    # how close is the produced output from being "real"?
    return torch.mean((D_out-1)**2)

def fake_mse_loss(D_out):
    # how close is the produced output from being "fake"?
    return torch.mean(D_out**2)

def cycle_consistency_loss(real_im, reconstructed_im, lambda_weight):
    # calculate reconstruction loss 
    # as absolute value difference between the real and reconstructed images
    reconstr_loss = torch.mean(torch.abs(real_im - reconstructed_im))
    # return weighted loss
    return lambda_weight*reconstr_loss
    
    

In [10]:
import torch.optim as optim

# hyperparams for Adam optimizer
lr=0.0002
beta1=0.5
beta2=0.999 # default value

g_params = list(G_XtoY.parameters()) + list(G_YtoX.parameters())  # Get generator parameters

# Create optimizers for the generators and discriminators
g_optimizer = optim.Adam(g_params, lr, [beta1, beta2])
d_x_optimizer = optim.Adam(D_X.parameters(), lr, [beta1, beta2])
d_y_optimizer = optim.Adam(D_Y.parameters(), lr, [beta1, beta2])

NameError: name 'G_XtoY' is not defined

In [30]:
#TRAINING LOOP


# train the network

def training_loop(dataloader_X, dataloader_Y, test_dataloader_X, test_dataloader_Y, 
                  n_epochs=1000):
    reconstructed_x_loss_min = math.inf
    reconstructed_y_loss_min = math.inf
    print_every=10
    
    # keep track of losses over time
    losses = []
    
    test_iter_X = iter(test_dataloader_X)
    test_iter_Y = iter(test_dataloader_Y)

    # Get some fixed data from domains X and Y for sampling. that allow us to inspect the model's performance.
    fixed_X = test_iter_X.next()[0]
    fixed_Y = test_iter_Y.next()[0]


    # batches per epoch
    iter_X = iter(dataloader_X)
    iter_Y = iter(dataloader_Y)
    batches_per_epoch = min(len(iter_X), len(iter_Y))

    for epoch in range(1, n_epochs+1):
        
        
        # Reset iterators for each epoch
        if epoch % batches_per_epoch == 0:
            iter_X = iter(dataloader_X)
            iter_Y = iter(dataloader_Y)

        data_X, _ = iter_X.next()
        data_Y, _ = iter_Y.next()
        
        # move images to GPU if available (otherwise stay on CPU)
        device = torch.device("cpu")
        data_X = data_X.to(device)
        data_Y = data_Y.to(device)
        
        data_X = data_X.permute(0,3,1,2).float()
        data_Y = data_Y.permute(0,3,1,2).float()

        # ============================================
        #            TRAIN THE DISCRIMINATORS
        # ============================================

        ##   First: D_X, real and fake loss components   ##

        # Train with real images
        d_x_optimizer.zero_grad()

        # 1. Compute the discriminator losses on real data from DOMAINN+ X
        out_x = D_X(data_X)
        D_X_real_loss = real_mse_loss(out_x)
        
        # Train with fake X domain generated from domain Y
        
        # 2. Generate fake data that look like domain X based on real data in domain Y
        fake_X = G_YtoX(data_Y)

        # 3. Compute the fake loss for D_X
        out_x = D_X(fake_X)
        D_X_fake_loss = fake_mse_loss(out_x)
        

        # 4. Compute the total loss and perform backprop
        d_x_loss = D_X_real_loss + D_X_fake_loss
        d_x_loss.backward()
        d_x_optimizer.step()

        
        ##   Second: D_Y, real and fake loss components   ##
        
        # Train with real data
        d_y_optimizer.zero_grad()
        
        # 1. Compute the discriminator losses on real data from DOMAIN Y
        out_y = D_Y(data_Y)
        D_Y_real_loss = real_mse_loss(out_y)
        
        # Train with fake images

        # 2. Generate fake data that look like domain Y based on real data in domain X
        fake_Y = G_XtoY(data_X)

        # 3. Compute the fake loss for D_Y
        out_y = D_Y(fake_Y)
        D_Y_fake_loss = fake_mse_loss(out_y)

        # 4. Compute the total loss and perform backprop
        d_y_loss = D_Y_real_loss + D_Y_fake_loss
        d_y_loss.backward()
        d_y_optimizer.step()


        # =========================================
        #            TRAIN THE GENERATORS
        # =========================================

        ##    First: generate fake X data and reconstructed Y data   ##
        g_optimizer.zero_grad()

        # 1. Generate fake images that look like domain X based on real images in domain Y
        fake_X = G_YtoX(data_Y)

        # 2. Compute the generator loss based on domain X
        out_x = D_X(fake_X)
        g_YtoX_loss = real_mse_loss(out_x)

        # 3. Create a reconstructed y
        # 4. Compute the cycle consistency loss (the reconstruction loss)
        reconstructed_Y = G_XtoY(fake_X)
        reconstructed_y_loss = cycle_consistency_loss(data_Y, reconstructed_Y, lambda_weight=10)


        ##    Second: generate fake Y images and reconstructed X images    ##

        # 1. Generate fake images that look like domain Y based on real images in domain X
        fake_Y = G_XtoY(data_X)

        # 2. Compute the generator loss based on domain Y
        out_y = D_Y(fake_Y)
        g_XtoY_loss = real_mse_loss(out_y)

        # 3. Create a reconstructed x
        # 4. Compute the cycle consistency loss (the reconstruction loss)
        reconstructed_X = G_YtoX(fake_Y)
        reconstructed_x_loss = cycle_consistency_loss(data_X, reconstructed_X, lambda_weight=10)

        # 5. Add up all generator and reconstructed losses and perform backprop
        g_total_loss = g_YtoX_loss + g_XtoY_loss + reconstructed_y_loss + reconstructed_x_loss
        g_total_loss.backward()
        g_optimizer.step()


        # Print the log info
        if epoch % print_every == 0:
            # append real and fake discriminator losses and the generator loss
            losses.append((d_x_loss.item(), d_y_loss.item(), g_total_loss.item()))
            print('Epoch [{:5d}/{:5d}] | d_X_loss: {:6.4f} | d_Y_loss: {:6.4f} | g_total_loss: {:6.4f}'.format(
                    epoch, n_epochs, d_x_loss.item(), d_y_loss.item(), g_total_loss.item()))

            
#         sample_every=100
        # Save the generated samples
#         if epoch % sample_every == 0:
#             G_YtoX.eval() # set generators to eval mode for sample generation
#             G_XtoY.eval()
#             save_samples(epoch, fixed_Y, fixed_X, G_YtoX, G_XtoY, batch_size=16)
#             G_YtoX.train()
#             G_XtoY.train()

        #uncomment these lines, if you want to save your model
#          checkpoint_every=100
#         # Save the model parameters
        if reconstructed_x_loss < reconstructed_x_loss_min:
            reconstructed_x_loss_min=reconstructed_x_loss
            checkpoint = {
                'epoch': epoch+1,
                'state_dict': G_XtoY.state_dict(),
                'optimizer': g_optimizer.state_dict(),
                'minxloss': reconstructed_x_loss_min
            }
            
            #save_ckp(checkpoint,True,"checkpointsGx","modelsGx")
            torch.save(checkpoint,"XtoY.pt")
            print("Generator X-Y saved with Cycle LOSS: ", reconstructed_x_loss_min)
            
            checkpoint = {
                'epoch': epoch+1,
                'state_dict': D_X.state_dict(),
                'optimizer': d_x_optimizer.state_dict(),
            }
            
            torch.save(checkpoint,"Dx.pt")
            #save_ckp(checkpoint,True,"checkpointsDx","modelsDx")
            
        
        if reconstructed_y_loss < reconstructed_y_loss_min:
            reconstructed_y_loss_min=reconstructed_y_loss
            checkpoint = {
                'epoch': epoch+1,
                'state_dict': G_YtoX.state_dict(),
                'optimizer': g_optimizer.state_dict(),
                'minxloss': reconstructed_y_loss_min
            }
            torch.save(checkpoint,"YtoX.pt")
            #save_ckp(checkpoint,True,"checkpointsGy","modelsGy")
            print("Generator Y-X saved with Cycle LOSS: ", reconstructed_y_loss_min)
            
            checkpoint = {
                'epoch': epoch+1,
                'state_dict': D_Y.state_dict(),
                'optimizer': d_y_optimizer.state_dict(),
            }
            torch.save(checkpoint,"Dy.pt")
            #save_ckp(checkpoint,True,"checkpointsDy","modelsDy")
            
            
        
            
        
            
                
        
             

    return losses


In [75]:
losses = training_loop(dataloader_X, dataloader_Y, test_dataloader_X, test_dataloader_Y, n_epochs=150)

Generator X-Y saved with Cycle LOSS:  tensor(0.0495, grad_fn=<MulBackward0>)
Generator Y-X saved with Cycle LOSS:  tensor(0.0589, grad_fn=<MulBackward0>)
Generator Y-X saved with Cycle LOSS:  tensor(0.0587, grad_fn=<MulBackward0>)
Generator Y-X saved with Cycle LOSS:  tensor(0.0571, grad_fn=<MulBackward0>)
Generator X-Y saved with Cycle LOSS:  tensor(0.0492, grad_fn=<MulBackward0>)
Generator Y-X saved with Cycle LOSS:  tensor(0.0552, grad_fn=<MulBackward0>)
Generator Y-X saved with Cycle LOSS:  tensor(0.0491, grad_fn=<MulBackward0>)
Generator X-Y saved with Cycle LOSS:  tensor(0.0404, grad_fn=<MulBackward0>)
Generator Y-X saved with Cycle LOSS:  tensor(0.0474, grad_fn=<MulBackward0>)
Epoch [   10/  150] | d_X_loss: 0.4827 | d_Y_loss: 0.4629 | g_total_loss: 0.7001
Generator Y-X saved with Cycle LOSS:  tensor(0.0415, grad_fn=<MulBackward0>)
Generator Y-X saved with Cycle LOSS:  tensor(0.0354, grad_fn=<MulBackward0>)
Generator X-Y saved with Cycle LOSS:  tensor(0.0384, grad_fn=<MulBackwar

In [73]:
print(losses)

[(0.5160729289054871, 0.4828607738018036, 0.9774311780929565), (0.5277869701385498, 0.49000129103660583, 0.93255615234375), (0.495054692029953, 0.4750649333000183, 0.9728460907936096), (0.49318191409111023, 0.48721519112586975, 0.8967345952987671), (0.4659627676010132, 0.4949831962585449, 0.9031214714050293), (0.5016170740127563, 0.4577341675758362, 0.849012017250061), (0.50105881690979, 0.46250981092453003, 0.897021472454071), (0.5115640163421631, 0.49374228715896606, 0.8116147518157959), (0.5122978091239929, 0.5086577534675598, 0.7753113508224487), (0.47132331132888794, 0.4540477395057678, 0.7623947858810425), (0.5150717496871948, 0.468146950006485, 0.7518098950386047), (0.5280783772468567, 0.4572400450706482, 0.7999032735824585), (0.48093318939208984, 0.5004737973213196, 0.7171396017074585), (0.47028985619544983, 0.4705091416835785, 0.7120597958564758), (0.5312702655792236, 0.4774635434150696, 0.7357356548309326)]


In [76]:
x = npy_loader("jazz.npy")
x = x.unsqueeze(0)
print(x.shape)


save_midis(x,"jazz.mid")

x = G_YtoX(x.permute(0,3,1,2).float())

save_midis(x.permute(0,2,3,1).detach().numpy(),"jazztoclassic.mid")


torch.Size([1, 64, 84, 1])


In [21]:
model = Generator(res_blocks=10)
optimizer = optim.Adam(model.parameters(), lr, [beta1, beta2])
ckp_path = "XtoY.pt"
model,_,_= load_ckp(ckp_path, model, optimizer)


In [23]:
x = npy_loader("classic.npy")
x = x.unsqueeze(0)
print(x.shape)

model.eval()
save_midis(x,"classicnew.mid")

x = model(x.permute(0,3,1,2).float())

save_midis(x.permute(0,2,3,1).detach().numpy(),"classicnewTOjazz.mid")

torch.Size([1, 64, 84, 1])
